# 1. Naive bayes classifier 배경

설명변수와 반응변수를 분리해 생각 

<br>

아래와 같은 상황일 때, 반응변수가 특정하게 발생하는 패턴이 자주 발생
- 특정 반응변수 자체가 많이 일어나는 확률
- 특정 반응변수가 먼저 발생하고, 해당 설명변수가 발생하는 확률이 높은 경우

<br>

Naive bayes는 `설명변수 간의 독립`을 가정 <br>
ex. 테니스를 쳤을 때, 겉보기 날씨와 습도는 서로 상관관계가 없음을 가정

**수학적 개념 - 조건부 확률**

B가 주어졌을 때 사건 A의 `조건부 확률` <br>
$ P(A|B) = \frac{P(A \cap B)}{P(B)} $

**베이즈정리**<br>
$ P(A_i|B) = \frac{P(B|A_i) * P(A_i)}{P(B)} $

- 특정 반응변수 자체가 많이 일어나는 확률 == $P(A_i)$
- 특정 반응변수가 먼저 발생하고, 해당 설명변수가 발생하는 확률이 높은 경우 == $P(B|A_i)$

# 2. Naive bayes classifier 종류

- Gaussian naive bayes classifier : 설명변수가 `연속형`인 경우
    - 설명변수를 정규분포로 가정

<br>

- Multinomial naive bayes classifier : 설명변수가 `범주형`인 경우
    - Bernoulli naive bayes classifier : 설명변수가 `이분형`인 경우

# 3. Gaussian Naive Bayes

## 3-1. 데이터, 모듈 불러오기

In [4]:
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
import pandas as pd

In [5]:
iris = datasets.load_iris()
df_X = pd.DataFrame(iris.data)
df_Y = pd.DataFrame(iris.target)

In [6]:
df_X.head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [7]:
df_Y.head()

,0
0,0
1,0
2,0
3,0
4,0


## 3-2. fitting

In [8]:
gnb = GaussianNB()
fitted = gnb.fit(iris.data, iris.target)
y_pred = fitted.predict(iris.data)

In [9]:
fitted.predict_proba(iris.data)[[1, 48, 51, 100]]

array([[1.00000000e+000, 1.51480769e-017, 2.34820051e-025],
       [1.00000000e+000, 2.63876217e-018, 2.79566024e-025],
       [7.27347795e-102, 9.45169639e-001, 5.48303606e-002],
       [3.23245181e-254, 6.35381031e-011, 1.00000000e+000]])

In [10]:
fitted.predict(iris.data)[[1, 48, 51, 100]]

array([0, 0, 1, 2])

## 3-3. confusion matrix

In [11]:
from sklearn.metrics import confusion_matrix

In [12]:
confusion_matrix(iris.target, y_pred)

array([[50,  0,  0],
       [ 0, 47,  3],
       [ 0,  3, 47]])

## 3-4. priors 설정

In [14]:
gnb2 = GaussianNB(priors = [1/100, 1/100, 98/100]) 
# 세번째 클래스의 출현확률을 높게 줘서 두번째 클래스임에도 세번째로 예측하거나
# 세번째 클래스는 모두 맞추는 것을 보여줌
fitted2 = gnb2.fit(iris.data, iris.target)
y_pred2 = fitted2.predict(iris.data)
confusion_matrix(iris.target, y_pred2)

array([[50,  0,  0],
       [ 0, 33, 17],
       [ 0,  0, 50]])

In [15]:
gnb3 = GaussianNB(priors = [1/100, 98/100, 1/100]) 
# 두번째 클래스의 출현 확률을 높게 줘서, 세번째 클래스를 두번째로 예측하는 경우 발생
fitted3 = gnb3.fit(iris.data, iris.target)
y_pred3 = fitted3.predict(iris.data)
confusion_matrix(iris.target, y_pred3)

array([[50,  0,  0],
       [ 0, 50,  0],
       [ 0, 14, 36]])

# 4. Multinomial naive bayes

## 4-1. 데이터, 모듈 설정

In [16]:
from sklearn.naive_bayes import MultinomialNB
import numpy as np

In [17]:
X = np.random.randint(5, size = (6, 100))
y = np.array([1, 2, 3, 4, 5, 6])

In [20]:
X

array([[2, 0, 3, 4, 0, 1, 2, 4, 1, 3, 3, 1, 3, 4, 1, 0, 1, 4, 3, 2, 4, 3,
        0, 1, 2, 1, 1, 3, 0, 3, 1, 4, 0, 0, 4, 0, 4, 3, 1, 3, 4, 3, 2, 0,
        1, 2, 0, 4, 1, 0, 3, 1, 1, 4, 0, 0, 1, 3, 3, 4, 1, 4, 4, 4, 2, 4,
        4, 2, 0, 3, 3, 0, 2, 4, 3, 4, 3, 0, 1, 0, 2, 1, 1, 0, 4, 1, 4, 2,
        2, 2, 2, 2, 4, 0, 0, 4, 0, 0, 1, 2],
       [0, 3, 2, 1, 1, 1, 1, 1, 2, 2, 4, 1, 1, 3, 1, 3, 2, 1, 1, 4, 3, 0,
        2, 3, 2, 0, 2, 4, 1, 0, 4, 1, 2, 4, 2, 3, 0, 3, 3, 4, 0, 1, 1, 2,
        4, 1, 0, 4, 0, 4, 0, 1, 3, 0, 0, 3, 4, 0, 4, 0, 3, 1, 3, 4, 1, 3,
        1, 3, 1, 2, 0, 4, 3, 4, 1, 3, 3, 2, 0, 3, 2, 2, 0, 4, 3, 2, 3, 1,
        4, 0, 1, 2, 0, 3, 4, 2, 2, 1, 0, 1],
       [3, 3, 2, 4, 3, 2, 2, 4, 3, 1, 3, 2, 2, 2, 2, 0, 2, 2, 1, 2, 0, 2,
        1, 0, 4, 0, 3, 1, 3, 3, 4, 2, 3, 1, 0, 2, 4, 3, 3, 3, 4, 2, 1, 3,
        0, 4, 1, 1, 2, 3, 2, 3, 2, 3, 0, 1, 2, 0, 3, 3, 2, 1, 2, 1, 4, 1,
        1, 1, 4, 0, 4, 4, 0, 1, 1, 3, 1, 1, 4, 3, 1, 2, 1, 4, 4, 3, 4, 1,
        2, 0, 0, 0, 4,

In [21]:
y

array([1, 2, 3, 4, 5, 6])

## 4-2. fitting

In [22]:
clf = MultinomialNB()
clf.fit(X, y)

MultinomialNB()

In [27]:
print(clf.predict(X[2:3]))

[3]


In [28]:
clf.predict_proba(X[2:3])

array([[5.05210960e-33, 2.30287116e-32, 1.00000000e+00, 1.08655594e-26,
        3.78668378e-32, 2.13161144e-32]])

In [29]:
clf.predict(X[2:3])

array([3])

## 4-3. priors 설정

In [38]:
clf2 = MultinomialNB(class_prior = [0.1, 0.5, 0.1, 0.1, 0.1, 0.1])
clf2.fit(X, y)

MultinomialNB(class_prior=[0.1, 0.5, 0.1, 0.1, 0.1, 0.1])

In [39]:
clf2.predict_proba(X[2:3])
# 두번째 클래스의 확률 값이 높아짐

array([[5.05210960e-33, 1.15143558e-31, 1.00000000e+00, 1.08655594e-26,
        3.78668378e-32, 2.13161144e-32]])